### 파일 경로 설정하기

In [1]:
from PIL import Image
import os,cv2, shutil
from tqdm.notebook import tqdm
from glob import glob
import numpy as np, pandas as pd

# train_img_path = f'D:\Real_dataset\ICDAR2015/' + 'Train_img_2015'
# train_gt_path = f'D:\Real_dataset\ICDAR2015/' + 'Train_gt_2015'
# test_img_path = f'D:\Real_dataset\ICDAR2015/' + 'Test_img_2015'
# test_gt_path = f'D:\Real_dataset\ICDAR2015/' + 'Test_gt_2015'

### 각 폴더별 파일이름들 list로 추출 후, 폴더 생성 & 파일 복사(원본은 그냥 두고 copy)

In [2]:
# os.makedirs('D:\Real_dataset\ICDAR2015/total_img',exist_ok=True)
# os.makedirs('D:\Real_dataset\ICDAR2015/total_gt',exist_ok=True)

# for img in tqdm(os.listdir(train_img_path)):
#     shutil.copy(f'{train_img_path}/{img}',f'D:\Real_dataset\ICDAR2015/total_img/{img}')

# for img in tqdm(os.listdir(test_img_path)):
#     shutil.copy(f'{test_img_path}/{img}',f"D:\Real_dataset\ICDAR2015/total_img/img_{int(img.split('_')[1].split('.')[0]) + 1000}.jpg")

# for gt in tqdm(os.listdir(train_gt_path)):
#     shutil.copy(f'{train_gt_path}/{gt}',f'D:\Real_dataset\ICDAR2015/total_gt/{gt}')

# for gt in tqdm(os.listdir(test_gt_path)):
#     shutil.copy(f'{test_gt_path}/{gt}',f"D:\Real_dataset\ICDAR2015/total_gt/gt_img_{int(gt.split('_')[2].split('.')[0]) + 1000}.txt")

### 이미지경로와 gt경로 합친 후, rect값,label값 추출
원본 이미지1장과 그 1장에 대한 textbox의 좌표값을 1세트로 묶어서 for문 실행

In [3]:
img_list = pd.DataFrame(glob('D:\Real_dataset\ICDAR2015/total_img/*'))
gt_list = pd.DataFrame(glob('D:\Real_dataset\ICDAR2015/total_gt/*'))

total_list = np.array(pd.concat([img_list,gt_list],axis=1))

def extract_vertices(lines):
	labels = []
	vertices = []
 
	for line in lines:
		vertices.append(list(map(int,line.rstrip('\n').lstrip('\ufeff').split(',')[:8])))
		labels.append(0 if '###' in line else line.split(',')[-1].rstrip('\n'))
  
	labels = pd.DataFrame(labels,columns=['labels'])
	vertices = pd.DataFrame({'vertices':vertices})
 
	infor = pd.concat([labels,vertices],axis=1)
 
	del_index = infor[infor['labels']==0].index
	infor.drop(del_index,inplace=True)
	infor = np.array(infor)
	
	return infor

### crop 진행 

In [4]:

for path in total_list:
    img_path = path[0]
    gt_path = path[1]
       
    with open(f'{gt_path}', 'r', encoding='UTF-8') as f:
        lines = f.readlines()
        
    infor = extract_vertices(lines)
    
    # 실질적으로 자르는 부분은 여기가 전부다.
    # 1장의 이미지를 imread로 읽고 그 이미지를 copy로 복사하고 원하는 부분의 x,y좌표를 찍어서
    # boundingRect로 그려서 imwrite로 저장
    crop_img = cv2.imread(img_path)

    for lv in infor:
        label = lv[0]
        vertice = lv[1]
        
        pts = np.array([ [vertice[0],vertice[1]], [vertice[2],vertice[3]], [vertice[4],vertice[5]], [vertice[6],vertice[7]] ])
        
        rect = cv2.boundingRect(pts)
        x,y,w,h = rect
        croped = crop_img[y:y+h, x:x+w].copy()

        cv2.imwrite(f"D:\Real_dataset\ICDAR2015\crop/{label}.png", croped)